In [1]:
import pandas as pd

df = pd.read_json("./IMDB_reviews.json", lines=True).drop_duplicates("review_text").sample(frac=1)
df.user_id = df.user_id.astype('category').cat.codes

In [2]:
df.is_spoiler.value_counts()

False    422529
True     150856
Name: is_spoiler, dtype: int64